## SMO

### 1. SVM 优化目标

SVM 优化目标是一个 QP(二次规化，Quadratic Progamming) 问题。其基本形式如下：

$$
\max_{\textbf{⍺}} W(\textbf{⍺}) = \sum_{i = 1}^{m}\alpha_i - \frac{1}{2}\sum_{i = 1}^{m}\sum_{j = 1}^{m}y_iy_j
\textbf{k}(\textbf{x}_i, \textbf{x}_j)\alpha_i\alpha_j
\\ s.t. \space 0 \leqslant \alpha_i \leqslant C, i = 1, .., m
\\ \sum_{i = 1}^{m}y_i\alpha_i = 0
\tag 1
$$

(1)可以通过 SMO 来求解。当上面有 QP 满足 KKT 条件，并且 $Q_{ij} = y_iy_j
\textbf{k}(\textbf{x}_i, \textbf{x}_j)$ 矩阵是半正定时，则我们所得的解就是最优解。

KKT 条件：

$$
\alpha_i = 0 	\Rightarrow y_i f(\textbf{x}_i) \geqslant 1 \\
0 < \alpha_i < C 	\Rightarrow y_i f(\textbf{x}_i) = 1 \\
\alpha_i = C 	\Rightarrow y_i f(\textbf{x}_i) \leqslant 1
\tag 2
$$


### 2. SMO 原理

SMO 的思想是将 QP 问题分解成若干个小 QP 问题(Sub-QP) 问题。而这些 Sub-QP 问题通过分析就可求得其解。 SMO 在稀疏数据上表现非常好，特别适合文本分类。

在执行 SMO 算法时，可以对每个样本验证 KKT 条件，如果条件满足则就把样本加进来直到收敛。

SMO 每次只选择 2 个拉格朗日乘子来进行一起优化。SMO 算法有三个要点：
1. 如何求解更新所选的拉格朗日乘子 $\alpha_1, \alpha_2$。
2. 如何选择两个拉格朗日乘子 $\alpha_1, \alpha_2$。
3. 如果更新 b.

#### 2.1 如何求解两个拉格朗日乘子 $\alpha_1, \alpha_2$

<img src="smo-alpha.png" width="300">

$$
{\alpha}_2^{new, clipped} =\begin{cases}
               H, \space {\alpha}_2^{new} \geqslant H \\
               {\alpha}_2^{new}, L < a_2^{new} < H \\
               L, \space {\alpha}_2^{new} \leqslant L \\
            \end{cases}
$$

其中：

$$
{\alpha}_2^{new} = {\alpha}_2^{old} - \frac{y_2(E_1 - E_2)}{\eta} \\
E_i = f^{old}(\textbf{x}_i) - y_i
$$

当 $y_i = y_j$ 时：
$$
L = max(0, {\alpha}_1^{old} + {\alpha}_2^{old} - C) \\
H = min(C, {\alpha}_1^{old} + {\alpha}_2^{old})
$$

当 $y_i \neq y_j$ 时：

$$
L = max(0, {\alpha}_2^{old} - {\alpha}_1^{old}) \\
H = min(C, C + {\alpha}_2^{old} - {\alpha}_1^{old})
$$

而别一个乘子的计算方法为：

$$
{\alpha}_1^{new} = {\alpha}_1^{old} + y_1 y_2({\alpha}_2^{old} - {\alpha}_2^{new, clipped})
$$

#### 2.2 启发式搜索 ${\alpha}_1, {\alpha}_2$

两个乘子的选择策略是不一样的。

第一个：寻找一个违反 KKT 条件的样本，如果违反了 KKT 条件，那么这个样本对就的 $\alpha_i$ 就可拿来做优化。我们也不必遍历所有样本，我们只需要找到 $ 0 < \alpha_i < C$ 的乘子，并且该乘子对应的样本违反了 KKT 条件。直到所有乘子都满足 KKT  条件，允许的误差是 $\epsilon$。

第二个：基于第一相样本的误差 $E_1$， 寻找一个令 $|E_2 - E_1|$ 最大的样本，这样更新的速度最快(big step)， 最大化步长。

#### 2.3 计算 b 及误差缓存更新

第一步都需要重新计算 b，这样才能让所有优化的样本满足 KKT 条件。我们需要计算两个 threshold , 即 b1 和 b2。然后根据条件来选择 b1, b2 或两个的平均值。
计算的公式：

$$
b_1 = E_1 + y_1(\alpha_1^{new} - \alpha_1^{old})\textbf{k}(\textbf{x}_1, \textbf{x}_1) + y_2(\alpha_2^{new,clipped} - \alpha_2^{old})\textbf{k}(\textbf{x}_1, \textbf{x}_2) + b^{old} \\
b_2 = E_2 + y_1(\alpha_1^{new} - \alpha_1^{old})\textbf{k}(\textbf{x}_1, \textbf{x}_2) + y_2(\alpha_2^{new,clipped} - \alpha_2^{old})\textbf{k}(\textbf{x}_2, \textbf{x}_2) + b^{old}
$$

选择的条件：

```python
if 0 < ⍺1 < C: 
    b = b1
else 0 < ⍺2 < C: 
    b = b2
else:
    b = (b1 + b2) / 2
```

误差缓存更新：

$$
E_k^{new} = E_k^{old} + y_1(\alpha_1^{new} - \alpha_1^{old})\textbf{k}(\textbf{x}_1, \textbf{x}_k) + y_2(\alpha_2^{new,clipped} - \alpha_2^{old})\textbf{k}(\textbf{x}_2, \textbf{x}_k) + b^{old} - b^{new}
$$




### SMO 优化

权重向量更新：
$$
\textbf{w}^{new} = \textbf{w}^{old} + y_1(\alpha_1^{new} - \alpha_1^{old})\textbf{x}_1 + y_2(\alpha_2^{new,clipped} - \alpha_2^{old})\textbf{x}_2
$$

### SMO 实现

输入：样本集 $X \in \mathbb{R}^ {m * n}$, 即共有 m 个样本，n 个特征。 $\textbf{y} \in \mathbb{R} ^{m*1}$。
一个值得注意的地方是：我们的 SVM, SMO 推荐中的向量 $\textbf{w}, \textbf{b}, \textbf{x}_i, \textbf{y}$ 都是列向量。所在计算时需注意 X 的每一行的转置才是对应的列向量。

如 $E_i$ 的计算公式推导：

$$
f(\textbf{x}_i) = \textbf{w}^T \textbf{x}_i + b \\
\textbf{w} = \sum_{i = 1}^{m} \alpha_i y_i \textbf{x}_i = X^T \textbf{⍺} \odot \textbf{y} \\
$$
所以有：
$$
\begin{align}
f(\textbf{x}_i) &= \textbf{w}^T \textbf{x}_i + b \\
&= (X^T \textbf{⍺} \odot \textbf{y})^T \textbf{x}_i + b \\
&= (\textbf{⍺} \odot \textbf{y})^T X \textbf{x}_i + b 
\end{align}
$$

最终 $E_i$ 的计算公式为:

$$
E_i = (\textbf{⍺} \odot \textbf{y})^T X \textbf{x}_i + b - y_i
$$